In [ ]:
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/notebooks/data/imdb_sample')

In [ ]:
def open_text(fn:PathOrStr):
    with open(fn,'r') as f: return ''.join(f.readlines())

In [ ]:
class Category(ItemBase):
    def __init__(self, idx, cat): self.data,self.cat = idx,cat
    def __str__(self):  return str(self.cat)

class CategoryList(ItemList):
    def __init__(self, items:Iterator, classes:Collection=None):
        super().__init__(items)
        if classes is None: classes = uniqueify(items)
        self.classes = classes
        self.class2idx = {v:k for k,v in enumerate(self.classes)}
        
    def new(self, items):
        return self.__class__(items, self.classes)
    
    def get(self, o): return Category(self.class2idx[o], o)

In [ ]:
class MultiCategory(Category):
    def __str__(self):  return ';'.join(map(str, self.cat))

class MultiCategoryList(CategoryList):
    def __init__(self, items:Iterator, classes:Collection=None, sep=None):
        if sep is not None: items = array(list(csv.reader(items, delimiter=sep)))
        if classes is None: classes = uniqueify(np.concatenate(items))
        super().__init__(items, classes)
    
    def get(self, o):
        return MultiCategory([self.class2idx[it] for it in o], o)

In [ ]:
def _treat_html(o:str)->str:
    return o.replace('\n','\\n')

def _text2html_table(items:Collection[Collection[str]], widths:Collection[int])->str:
    html_code = f"<table>"
    for w in widths: html_code += f"  <col width='{w}%'>"
    for line in items:
        html_code += "  <tr>\n"
        html_code += "\n".join([f"    <th>{_treat_html(o)}</th>" for o in line if len(o) >= 1])
        html_code += "\n  </tr>\n"
    return html_code + "</table>\n"

In [ ]:
class Text(ItemBase):
    def __init__(self, ids, text): self.data,self.text = ids,text
    def __str__(self):  return str(self.text)
    
    def show_batch(self, idxs:Collection[int], rows:int, ds:Dataset, figsize:Tuple[int,int]=(9,10))->None:
        from IPython.display import clear_output, display, HTML
        items = [['text', 'label']]
        for i in idxs[:rows]:
            x,y = ds[i]
            items.append([str(x), str(y)])
        display(HTML(_text2html_table(items, [90,10])))

class NumericalizedTextList(ItemList):
    def __init__(self, items:Iterator, vocab:Vocab=None, create_func:Callable=None, path:PathOrStr='.'):
        super().__init__(items, create_func, path)
        self.vocab = vocab
        
    def new(self, items):
        return self.__class__(items, self.vocab)
    
    def get(self, o): return Text(o, self.vocab.textify(o))
    
class TokenizedTextList(NumericalizedTextList):
    
    def preprocess(self, vocab:Vocab=None, max_vocab:int=60000, min_freq:int=2):
        self.vocab = ifnone(vocab, Vocab.create(self.items, max_vocab, min_freq))
        self.preprocess_kwargs = {'vocab': self.vocab}
        self.items = np.array([self.vocab.numericalize(t) for t in self.items])
    
class TextList(TokenizedTextList):
    def preprocess(self, tokenizer:Tokenizer=None, chunksize:int=10000, vocab:Vocab=None, 
                 max_vocab:int=60000, min_freq:int=2):
        tokenizer = ifnone(tokenizer, Tokenizer())
        tokens = []
        for i in progress_bar(range(0,len(self.items),chunksize), leave=False):
            tokens += tokenizer.process_all(self.items[i:i+chunksize])
        self.items = tokens
        super().preprocess(vocab, max_vocab, min_freq)
    
class TextFilesList(TextList):
    def __init__(self, items:Iterator, create_func:Callable=None, path:PathOrStr='.'):
        texts = [open_text(fn) for fn in items]
        super().__init__(texts, create_func, path)

In [ ]:
il = TextList.from_csv(path, 'texts.csv', create_func=None, col='text')

In [ ]:
ll = il.label_from_df(CategoryList, cols=0)

In [ ]:
sd = ll.random_split_by_pct()

In [ ]:
sd = sd.preprocess()

In [ ]:
data = sd.databunch()

In [ ]:
data.show_batch()

text,label
"sam mraovich should never be allowed to touch a camera again . if he does he should be arrested on the spot ... at the very least for petty xxunk . anybody who pays even a dime to rent any of his garbage should file a claim and be xxunk . this was innocently my first viewing of his "" xxunk ... and it will my last . ed wood looks awfully good to me right now . \n\n when i return this piece of crap to the video store , i will personally ask that it be taken off the shelf . an active xxunk of gay cinema , i am xxunk and xxunk that this warped , xxunk xxunk man - child be allowed to xxunk and xxunk something like this , with a xxunk pretty boy on the cover ( jamie xxunk xxunk , who , thankfully , has no other acting credits in imdb ) and an interesting synopsis on the back used as bait , and then market it as a "" movie "" rental . trust me , this has no place being on any rack anywhere ; it is simply not a movie in any sense of the word . offensive , irresponsible junk such as this can only be detrimental to the efforts being made to promote and support gay cinema ( hell , gay rights in general ! ) for those xxunk to rent this out because of the cover , you xxup will be disappointed . xxunk is not as xxunk to look at on film as he is on the cover , and he appears once or twice without a shirt -- that 's it . instead , the xxunk xxunk xxunk on us his own disgusting , sorry - looking xxunk - boy xxunk . \n\n this "" thing "" he "" xxunk "" is a xxunk xxunk project for mraovich . both he and his friend michael xxunk ( who , i understand , puts out similar sleazy garbage ) are terrible in this . mraovich is xxunk posing as a complete no - talent ( in all fields ) , desperate to grab onto any "" loser "" attention he can for himself . he is to be xxunk .",negative
"director / writer michael winner 's feature is a better than expected xxunk supernatural horror film ( although still schlock xxunk xxunk for ) , which really does by go xxunk . sure it might borrow ideas from other similar xxunk horror movies of this period , but still manages to bring its own psychological xxunk to the xxunk material ( of good vs. evil ) and a unique vision that has a fair share of impressively xxunk , if somewhat xxunk set - pieces . as a whole it 's xxunk , however remains intriguing by xxunk an ominous charge without going gang - xxunk with the scares . actually there 's always something going on amongst its busy xxunk , but it 's rather down - played with its xxunk xxunk to xxunk patterns and atmospheric xxunk , up until its vividly xxunk and grisly climax with a downbeat revelation . winner 's dressed up xxunk might feel xxunk , however it 's the ensemble cast that really holds it together as you try to spot the faces . there 's plenty too . some having more to do with the scheme of things than others , but there 's no doubts every one of them are committed , despite the xxunk crude nature of it all . it 's interesting to see names like xxunk miles ( who 's significantly creepy ! ) , beverly xxunk ( likewise ) , xxunk xxunk , xxunk xxunk , christopher walken , william xxunk ( a neat xxunk ) , jeff goldblum , jerry orbach and tom xxunk in bit parts . then you got a mild - mannered chris xxunk and xxunk gorgeous cristina raines in the leads . offering able support xxunk xxunk , martin xxunk , xxunk xxunk , john carradine , xxunk xxunk and arthur kennedy . the script does throw around many characters , as well as notions but gets xxunk xxunk by trying to xxunk all of it in . however it 's xxunk air works in its favour in establishing the xxunk and xxunk of what 's really going on here . is there a reason for all of this , and why is it surrounding raines ' character ? the emphasis is mainly built upon that moody angle , as it begins to slowly shed light of her inner xxunk and that of the strange / worrying experiences she encounters when she 's moves into her new apartment . this is where winner tries to pull out the xxunk shades , which pr